In [90]:
import tools, timeit, os
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from urllib.parse import urlencode
%config InlineBackend.rc = {}
%matplotlib inline

In [91]:
# Constants
DIR_DATA = '../data/PTF/'

In [216]:
# URLs
CATALOG_API = 'http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query'
CALI_IMAGES_CATALOG_API = 'http://irsa.ipac.caltech.edu/ibe/search/ptf/images/level1'
CALI_IMAGES_API = 'http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level1'
REF_IMAGES_CATALOG_API = 'http://irsa.ipac.caltech.edu/ibe/search/ptf/images/level2'
REF_IMAGES_API = 'http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level2'

# Download 20000 Transients

#### Create request

In [99]:
query = dict()
query['catalog'] = 'ptf_objects'
query['spatial'] = 'None'
query['selcols'] = 'ra,dec,oid,nobs,ngoodobs'
query['outfmt'] = '1'
query['outrows'] = '20000'
query['constraints'] = 'transient_flag=1 AND nobs >= 20 AND ngoodobs>= 15'

query_url = CATALOG_API + '?' + urlencode(query)
print(query_url)

http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?spatial=None&outfmt=1&selcols=ra%2Cdec%2Coid%2Cnobs%2Cngoodobs&catalog=ptf_objects&constraints=transient_flag%3D1+AND+nobs+%3E%3D+20+AND+ngoodobs%3E%3D+15&outrows=20000


#### Download data

In [ ]:
output_directory = DIR_DATA + 'cats/transients'
file_path = tools.download_file(query_url, 'objcat.tbl', output_directory)

In [101]:
file_path

'../data/PTF/cats/transients/objcat.tbl'

### Read catalogue

In [102]:
transient_df = Table.read(file_path, format='ascii').to_pandas()

In [103]:
transient_df.head()

,ra,dec,clon,clat,oid,nobs,ngoodobs
0,291.823749,45.201545,19h27m17.70s,45d12m05.56s,46572075001313,161,160
1,291.808502,45.210343,19h27m14.04s,45d12m37.23s,46572075001316,254,74
2,291.837458,45.231821,19h27m20.99s,45d13m54.56s,46572075001274,232,48
3,291.848750,45.202769,19h27m23.70s,45d12m09.97s,46572075001312,188,187
4,291.805963,45.177574,19h27m13.43s,45d10m39.27s,46572075002105,128,35


# Download R light curves

In [104]:
output_directory = DIR_DATA + 'light/R/'

#### Create request

In [129]:
def light_url(ra, dec):
    query = dict()
    query['catalog'] = 'ptf_lightcurves'
    query['spatial'] = 'None'
    query['selcols'] = 'oid,obsmjd,mag_autocorr'
    query['outfmt'] = '1'
    query['constraints'] = 'ra={} AND dec={} AND fid=2 AND goodflag=1'.format(ra, dec)
    query['order'] = 'obsmjd'
    
    query_url = CATALOG_API + '?' + urlencode(query)
    return query_url

#### Download Files

In [ ]:
for i, row in transient_df.iterrows():
    objid = row['oid']
    query_url = light_url(row['ra'], row['dec'])
    file_path = tools.download_file(query_url, '{}.tbl'.format(objid), output_directory)

# Download Calibrated Images 

### First, download the calibrated image catalogues

In [271]:
output_directory = DIR_DATA + 'cats/calibrated/'

#### Create request

In [272]:
def calibrated_catalog_url(ra, dec):
    query = dict()
    # Available Columns: 
    # http://irsa.ipac.caltech.edu/ibe//search/ptf/images/level1?FORMAT=METADATA&ct=html
    query['POS'] = '{},{}'.format(ra,dec)
    query['columns'] = 'pfilename,obsdate'
    query['where'] = 'filter=\'R\''
    # Only querying for the most centered image data
    query['mcen'] = '1'
    
    query_url = CALI_IMAGES_CATALOG_API + '?' + urlencode(query)
    return query_url

#### Download

In [ ]:
for i, row in transient_df.iterrows():
    if i == 2: break
    objid = row['oid']
    query_url = calibrated_catalog_url(row['ra'], row['dec'])
    file_path = tools.download_file(query_url, '{}.tbl'.format(objid), output_directory)

### Next, download the associated images

In [274]:
output_directory = DIR_DATA + 'images/calibrated/'
IMAGE_SIZE = (50,50) # arcsec

In [275]:
def calibrated_image_url(ra, dec, image_url_path, size):
    query = dict()
    # Available Columns: 
    query['center'] = '{},{}'.format(ra, dec)
    query['size'] = '{},{}arcsec'.format(size[0], size[1])
    query_url = CALI_IMAGES_API + '/' + image_url_path + '?' + urlencode(query)
    return query_url

In [ ]:
for i, row in transient_df.iterrows():
    if i == 2: break
    calibrated_catalog_path = DIR_DATA + 'cats/calibrated/{}.tbl'.format(row['oid'])
    calibrated_df = Table.read(calibrated_catalog_path, format='ascii').to_pandas()
    query_url = calibrated_image_url(row['ra'], row['dec'], calibrated_df.iloc[0]['pfilename'], IMAGE_SIZE)
    print(query_url)
    file_path = tools.download_file(query_url, '{}.fits'.format(row['oid']), output_directory)

# Download Reference Images 

### First, download the reference image catalogues

In [277]:
output_directory = DIR_DATA + 'cats/reference/'

#### Create request

In [278]:
def reference_catalog_url(ra, dec):
    query = dict()
    # Available Columns: 
    # http://irsa.ipac.caltech.edu/ibe//search/ptf/images/level1?FORMAT=METADATA&ct=html
    query['POS'] = '{},{}'.format(ra,dec)
    query['columns'] = 'filename'
    query['where'] = 'fid=2'
    # Only querying for the most centered image data
    query['mcen'] = '1'
    
    query_url = REF_IMAGES_CATALOG_API + '?' + urlencode(query)
    return query_url

#### Download

In [ ]:
for i, row in transient_df.iterrows():
    if i == 2: break
    query_url = reference_catalog_url(row['ra'], row['dec'])
    print(query_url)
    file_path = tools.download_file(query_url, '{}.tbl'.format(row['oid']), output_directory)

### Next, download the associated images

In [280]:
output_directory = DIR_DATA + 'images/reference/'

##### Full reference image paths are hidden, so they need to be extracted from filename

In [281]:
def extract_ref_image_path(filename):
    words = filename.split('_')
    f1 = words[1][:4]
    f2 = words[1]
    f3 = words[2][0] + words[2][1:].lstrip('0')
    if len(f3) == 1: f3 += '0'
    f4 = words[3][0] + words[3][1:].lstrip('0')
    if len(f4) == 1: f4 += '0'
    return '{}/{}/{}/{}/{}'.format(f1,f2,f3,f4,filename)

In [282]:
def reference_image_url(ra, dec, image_url_path, size):
    query = dict()
    # Available Columns: 
    query['center'] = '{},{}'.format(ra, dec)
    query['size'] = '{},{}arcsec'.format(size[0], size[1])
    query_url = REF_IMAGES_API + '/' + image_url_path + '?' + urlencode(query)
    return query_url

In [ ]:
for i, row in transient_df.iterrows():
    if i == 2: break
    reference_catalog_path = DIR_DATA + 'cats/reference/{}.tbl'.format(row['oid'])
    reference_df = Table.read(reference_catalog_path, format='ascii').to_pandas()
    print(reference_df.iloc[0])
    ref_image_path = extract_ref_image_path(reference_df.iloc[0]['filename'])
    query_url = reference_image_url(row['ra'], row['dec'], ref_image_path, IMAGE_SIZE)
    print(query_url)
    file_path = tools.download_file(query_url, '{}.fits'.format(row['oid']), output_directory)

In [ ]:
PTF_d004657_f01_c06_u000150682_p12_psfcat.ctlg